In [3]:
import pandas as pd

In [4]:
data_train = pd.read_csv('./train.tsv',sep='\t')
data_test = pd.read_csv('./test.tsv',sep='\t')

In [5]:
data_train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [6]:
data_train.shape

(156060, 4)

In [7]:
data_test.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [8]:
data_test.shape

(66292, 3)

In [9]:
# 取出train評論內容
train_sentences = data_train['Phrase']

# 取出test評論內容
test_sentences = data_test['Phrase']

# 用concat將兩個和在一起
sentences = pd.concat([train_sentences,test_sentences])

In [10]:
sentences.shape

(222352,)

In [11]:
label = data_train['Sentiment']

In [17]:
label.shape

(156060,)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1,4),
    max_features=150000
)

In [22]:
tf.fit(sentences)

TfidfVectorizer(max_features=150000, ngram_range=(1, 4))

In [23]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(train_sentences,label,random_state=1234)

In [24]:
x_train = tf.transform(x_train)
x_test = tf.transform(x_test)

In [25]:
x_train[1]

<1x150000 sparse matrix of type '<class 'numpy.float64'>'
	with 14 stored elements in Compressed Sparse Row format>

In [53]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(x_train,y_train)
print('TF-IDF，使用sklearn單純貝氏分類，準確率:',classifier.score(x_test,y_test))

TF-IDF，使用sklearn單純貝氏分類，準確率: 0.6045367166474432


In [29]:
test_X = tf.transform(data_test['Phrase'])

In [45]:
predictions = classifier.predict(test_X)

In [46]:
predictions

array([2, 2, 2, ..., 2, 2, 2], dtype=int64)

In [47]:
predictions.shape

(66292,)

In [48]:
data_test.loc[:,'Sentiment'] = predictions

In [49]:
data_test.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,156061,8545,An intermittently pleasing but mostly routine ...,2
1,156062,8545,An intermittently pleasing but mostly routine ...,2
2,156063,8545,An,2
3,156064,8545,intermittently pleasing but mostly routine effort,2
4,156065,8545,intermittently pleasing but mostly routine,2


In [50]:
final_data = data_test.loc[:,['PhraseId','Sentiment']]

In [51]:
final_data.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


In [52]:
final_data.to_csv('final_data.csv',index=None)